# Extract uniform Mid Features
This script selected the mid window size so that every audio clip had the same number of mid-term  features, regardless of the length of the clip.

We never came back to this idea, but we keep it for reference.

In [ ]:
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import MidTermFeatures as mF
from pyAudioAnalysis import audioBasicIO as aIO 

import numpy as np
import pandas as pd

import os
import time


import matplotlib.pyplot as plt
import seaborn as sns

files = os.listdir('../Data/Train_final/')
df_cats = pd.read_csv('../Data/Categories_train.csv')

# Sort files and df_cats
files.sort()
df_cats = df_cats.sort_values(by='FileID', ascending=True)

In [ ]:
def display_time(seconds):
    if seconds <= 60:
        print('Duration: %f (s)' %seconds)
    elif 60 < seconds <= 3600:
        print('Duration: %f (min)' %(seconds/60))
    else:
        print('Duration: %f (h)' %(seconds/3600))

Load the clips and compute the length

In [ ]:
time_start = time.time()

# Load audio files and calculate the longest length
Fs = []
Signals = []
df_cats['Length'] = 0
for file in files:
    fs, s = aIO.read_audio_file('../Data/Train_final/%s' %file)
    Fs.append(fs)
    Signals.append(s)
    
    # Compute the length of the clip
    df_cats.loc[df_cats.FileID==file.split('.')[0], 'Length'] = len(s)/fs

print('Number of files: %i' %(len(files)))
time_end = time.time()
display_time(time_end-time_start)

## Extract features

Select the number of midFeatures to extract, and calculate the window size to get that number of features.

Possibly because of approximation errors, sometimes we don't get the correct number of features. If that's the case, we either add or subtract `delta` from `mid_step` until we get the correct number of features.

There is a small technical issue, though. Sometimes, we get more features than we need when we add `delta` to `mid_step`, but less features when we subtract `delta`. For that reason, we also modify `delta` as we go. If we add and then subtract `delta` in consecutive steps, we divide `delta` by 2. This algorithm is inspired by midpoint optimization.

In [ ]:
nMidFeatures = 5

# Select short window and step size
s_win  = 0.050
s_step = 0.025
df_cats['mid_win'] = 0
df_cats['mid_step'] = 0

time_start = time.time()
for i in range(len(files)):
# for i in range(100):
    file = files[i].split('.')[0]
    s = Signals[i]
    fs = Fs[i]

    # Length of the clip
    L = len(s)/fs

    # Obtain nMidFeatures steps
    mid_step = L/nMidFeatures
    mid_win = 2*mid_step

    # Extract mid and short term features
    correct_size = False
    increasing = True
    while not correct_size:
        mt, st, mt_names = mF.mid_feature_extraction(s, fs,
                                                     fs*mid_win, fs*mid_step, 
                                                     fs*s_win,   fs*s_step)
        
        
        delta = 0.05
        # If we have the correct number of features, we're done
        if mt.shape[1] == nMidFeatures:
            correct_size = True
        
        # If not, we change the window size until we get the right number of features
        elif mt.shape[1] > nMidFeatures:
            print('Problem: %i' %(i))
            print('Size: %i' %(mt.shape[1]))
            
            # If the last step decreased delta, we divide delta by 2
            if not increasing:
                delta /= 2
                increasing = True
            
            mid_step += delta
            mid_win = 2*mid_step
        elif mt.shape[1] < nMidFeatures:
            print('Problem: %i' %(i))
            print('Size: %i' %(mt.shape[1]))
            
            # If the last step increased delta, we divide delta by 2
            if increasing:
                delta /= 2
                increasing = False
            
            mid_step -= delta
            mid_win = 2*mid_step
    
    if i%100==0:
        print('---------------------------')
        print(i)
        print('---------------------------')

    # Create dataframes in the first iteration
    if i==0:
        # Names for short term features
        st_names = ['_'.join(name.split('_')[:-1]) for name in mt_names[:68]]

        # Dataframe to hold mid term features
        df_mt = pd.DataFrame()
    
    # Save short term features to disk
    df_short = pd.DataFrame(data=st.T, columns=st_names)
    df_short.to_csv('Features_Unif_Length/%s_short.csv' %file, index=False)

    # Save mid term features to the dataframe
    # Rename columns to col_1, ..., col_5
    df_temp = pd.DataFrame(mt.T, columns=mt_names)
    df_temp.index = df_temp.index + 1
    df_temp = df_temp.stack()
    df_temp.index = df_temp.index.map('{0[1]}_{0[0]}'.format)
    df_temp = df_temp.to_frame().T
    
    # Also save the time windows
    df_temp.insert(0, 'FileID', file)
    df_temp.insert(1, 'Length', L)
    df_temp.insert(2, 'mid_win', mid_win)
    df_temp.insert(3, 'mid_step', mid_step)
    
    df_mt = pd.concat((df_mt, df_temp))

time_end = time.time()
display_time(time_end-time_start)

In [ ]:
df_mt

In [ ]:
df_cats

In [ ]:
df_mt.to_csv('Features/MidLength_Uniform.csv', index=False)

## Visualization
Try to visualize features

In [ ]:
emotions = np.unique(df_cats.Emotion)

Feature = 'energy_mean'
columns = ['%s_%i' %(Feature, i) for i in range(1,nMidFeatures)]

# Obtain a feature vector for every entry
sub_df = df_mt.loc[:,['FileID']+columns]

print(sub_df)

In [ ]:
for emotion in emotions:
    sub_df = df_mt.loc[cats_df.Emotion==emotion,columns]
    sns.
    plt.plot(np.arange(1,nMidFeatures), sub_df.values[:,1:].T);